In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import pandasql as ps
import pickle
import time

# from keras.datasets import reuters
# from keras.wrappers.scikit_learn import KerasClassifier
# from keras.layers.wrappers import TimeDistributed
# from keras.models import load_model, Sequential
# from keras.layers.embeddings import Embedding
# from keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional
# from keras.layers import Convolution1D, MaxPooling1D, Flatten
# from keras.utils import np_utils
# from keras.preprocessing.text import Tokenizer, one_hot, text_to_word_sequence
# from keras.preprocessing import sequence

# from keras.utils.visualize_util import plot
# from IPython.display import SVG
# from keras.utils.visualize_util import model_to_dot

# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder

# import custom code
import os
import sys
pardir = os.path.abspath(os.path.join(os.getcwd(), '../'))
script_path = pardir + "/2_common_aux_script"
print('Importing process_string.py \nfrom ' + script_path + " ...\n")
sys.path.append(script_path)
from process_string import process_string
sys.path.remove(script_path)


print(sys.version)

Importing process_string.py 
from /Users/altay.amanbay/Desktop/new node booster/experiments/3a.1 - Nets train/5 train model - keras (unigrams)/2_common_aux_script ...

3.5.2 |Anaconda 4.2.0 (x86_64)| (default, Jul  2 2016, 17:52:12) 
[GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)]


In [2]:
df = pd.read_csv('whole set train predictions/truth_and_predictions.csv')
df.head()

,description_mod1,category_id_mod1,category_full_path_mod1,target_le,Predictions_le,Predictions
0,it jeans maternity skinny jeans dark wash m,100,Apparel & Accessories > Apparel > Maternity,27,27,Apparel & Accessories > Apparel > Maternity
1,1822 denim butter maternity skinny jeans rinse...,100,Apparel & Accessories > Apparel > Maternity,27,27,Apparel & Accessories > Apparel > Maternity
2,25 j brand maternity skinny jean nirvana blue,100,Apparel & Accessories > Apparel > Maternity,27,27,Apparel & Accessories > Apparel > Maternity
3,26 j brand maternity skinny jean nirvana blue,100,Apparel & Accessories > Apparel > Maternity,27,27,Apparel & Accessories > Apparel > Maternity
4,26 james jeans maternity skinny external mater...,100,Apparel & Accessories > Apparel > Maternity,27,27,Apparel & Accessories > Apparel > Maternity


In [77]:
#1
# Find descriptions with multiple categories assigned
qry1 = """
        select target_le,category_full_path_mod1 from
        (
        select target_le,category_full_path_mod1, Predictions_le, count(*)
        from df 
        group by target_le, Predictions_le
        ) 
        group by target_le,category_full_path_mod1, category_full_path_mod1
        having count(target_le) > 1
        order by target_le, Predictions_le
      """

overlap_categories_df = ps.sqldf(qry1, globals())
print(overlap_categories_df.shape)

qry2 = """
        select df.category_full_path_mod1, df.category_id_mod1, df.target_le, df.Predictions, count(*)
        from df
        left join overlap_categories_df
        on df.target_le = overlap_categories_df.target_le
        where overlap_categories_df.target_le is not null
        group by df.category_full_path_mod1,df.category_id_mod1,df. Predictions
        order by df.category_full_path_mod1, df.Predictions
      """
overlap_descriptions_df = ps.sqldf(qry2, globals())
print(overlap_descriptions_df.shape)

(321, 2)
(8809, 5)


In [78]:
# List of non-accurate target_le
i  = 0
print(overlap_categories_df.loc[10*i:10*i+10,:])

    target_le                            category_full_path_mod1
0           0  Apparel & Accessories > Accessories > Belts & ...
1           1  Apparel & Accessories > Accessories > Belts & ...
2           2  Apparel & Accessories > Accessories > Belts & ...
3           3  Apparel & Accessories > Accessories > Belts & ...
4           4  Apparel & Accessories > Accessories > Belts & ...
5           8  Apparel & Accessories > Accessories > Ties > Boys
6           9   Apparel & Accessories > Accessories > Ties > Men
7          10  Apparel & Accessories > Accessories > Ties > O...
8          11      Apparel & Accessories > Apparel > Baby > Boys
9          12     Apparel & Accessories > Apparel > Baby > Girls
10         13     Apparel & Accessories > Apparel > Baby > Other


In [72]:
# Unroll certain non-accurate target_le with overlapping predictions
overlap_categories_df.head(10)
overlap_descriptions_df.loc[overlap_descriptions_df.target_le == 0,:].head(190)

,category_full_path_mod1,category_id_mod1,target_le,Predictions,count(*)
0,Apparel & Accessories > Accessories > Belts & ...,32,0,Apparel & Accessories > Accessories > Belts & ...,447
1,Apparel & Accessories > Accessories > Belts & ...,32,0,Apparel & Accessories > Accessories > Belts & ...,9
2,Apparel & Accessories > Accessories > Belts & ...,32,0,Apparel & Accessories > Accessories > Belts & ...,27
3,Apparel & Accessories > Accessories > Belts & ...,32,0,Apparel & Accessories > Accessories > Belts & ...,11
4,Apparel & Accessories > Accessories > Belts & ...,32,0,Apparel & Accessories > Accessories > Belts & ...,10
5,Apparel & Accessories > Accessories > Belts & ...,32,0,Apparel & Accessories > Accessories > Ties > Boys,1
6,Apparel & Accessories > Accessories > Belts & ...,32,0,Apparel & Accessories > Apparel > Jackets & Co...,1
7,Apparel & Accessories > Accessories > Belts & ...,32,0,Apparel & Accessories > Apparel > Leggings > W...,1
8,Apparel & Accessories > Accessories > Belts & ...,32,0,Apparel & Accessories > Apparel > Tops & Tees ...,4
9,Apparel & Accessories > Accessories > Belts & ...,32,0,Apparel & Accessories > Apparel > Tops & Tees ...,1


In [75]:
columns_ls = ['description_mod1','category_full_path_mod1','Predictions_le','Predictions']
df.loc[(df.target_le == 0) & (df.target_le != df.Predictions_le),columns_ls]

,description_mod1,category_full_path_mod1,Predictions_le,Predictions
319523,aquarius boys 2 for 1 boys,Apparel & Accessories > Accessories > Belts & ...,47,Apparel & Accessories > Apparel > Tops & Tees ...
319525,body strenth elastic suspenders for kids boys ...,Apparel & Accessories > Accessories > Belts & ...,2,Apparel & Accessories > Accessories > Belts & ...
319543,boys braided belt brown 814,Apparel & Accessories > Accessories > Belts & ...,2,Apparel & Accessories > Accessories > Belts & ...
319546,brooks brothers mens milano,Apparel & Accessories > Accessories > Belts & ...,49,Apparel & Accessories > Apparel > Tops & Tees ...
319574,carhartt big boys acrylic,Apparel & Accessories > Accessories > Belts & ...,14,Apparel & Accessories > Apparel > Jackets & Co...
319599,dinosaur childrens belt 2t,Apparel & Accessories > Accessories > Belts & ...,1,Apparel & Accessories > Accessories > Belts & ...
319630,elenker solid color kids baby elastic adjustab...,Apparel & Accessories > Accessories > Belts & ...,2,Apparel & Accessories > Accessories > Belts & ...
319647,kids 1 snap on plain leather belt,Apparel & Accessories > Accessories > Belts & ...,1,Apparel & Accessories > Accessories > Belts & ...
319650,kids 1 snap on punk rock red star studded leat...,Apparel & Accessories > Accessories > Belts & ...,1,Apparel & Accessories > Accessories > Belts & ...
319664,kids leather anchor belt medium warm brown,Apparel & Accessories > Accessories > Belts & ...,2,Apparel & Accessories > Accessories > Belts & ...


### Evaluation (optional)

In [75]:
# Manual check which nodes are not distinguishable for nets
class_ = 392
print(le.inverse_transform(class_))
print(X_test_ls[np_utils.categorical_probas_to_classes(y_test_ary)==class_],'\n')

class_ = 390
print(le.inverse_transform(class_))
print(X_test_ls[np_utils.categorical_probas_to_classes(y_test_ary)==class_],'\n')

class_ = 389
print(le.inverse_transform(class_))
print(X_test_ls[np_utils.categorical_probas_to_classes(y_test_ary)==class_],'\n')


Tools & Home Improvement > Lighting, Light Bulbs & Ceiling Fans > Other
[ 'progress lighting p299281 archiethree light bath vanity antique nickel finish'
 'bulbrite 60a15f 60watt incandescent a15 appliance bulb frost'
 'greenhouse indooroutdoor chandelier rasped iron finish' ...,
 'ge advantage fluorescent'
 'feit ctcdm500led 60w equivalent candelabra base torpedo tip chandelier led light soft white'
 'alena 97 arched floor lamp by house of hampton'] 

Tools & Home Improvement > Lighting, Light Bulbs & Ceiling Fans > Light Fixtures & Lamps
['cortina nightstand right door'
 'camino vintage candelabra twotier chandelier 72'
 'golden lighting 2501ba3' ..., 'aspect white 23 open unit'
 'harpwell 7light oilrubbed bronze chandelier'
 'connie 2light antique black flush mount'] 

Tools & Home Improvement > Lighting, Light Bulbs & Ceiling Fans > Light Bulbs
[ '3 pack led light bulbs lohas b35 7w soft white 3000k e12 candelabra bulb equivalent to 6065 watt incandescent'
 'br30 led bulbsluminwiz 

In [61]:
#pd.crosstab(pd.Series(y_test_ary.ravel()), pd.Series(predictions_rnd.ravel()), rownames=['True'], colnames=['Predicted'], margins=True)
pd.crosstab(pd.Series(np_utils.categorical_probas_to_classes(y_test_ary)), pd.Series(predictions), rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,1,2,3,4,5,6,9,10,11,12,...,389,390,392,393,394,395,396,397,403,All
True,,,,,,,,,,,,,,,,,,,,,
1,107,0,24,4,4,0,5,1,0,0,...,0,0,0,0,0,0,0,0,0,160
2,7,12,0,0,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,32
3,21,1,1126,120,148,0,0,8,0,0,...,0,0,0,0,0,0,0,0,0,1490
4,3,3,275,826,254,0,0,12,0,0,...,0,0,0,0,0,0,0,0,0,1424
5,5,1,104,84,1242,0,0,2,0,0,...,0,1,0,0,1,0,0,0,0,1508
6,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
9,0,0,0,0,0,0,304,49,2,0,...,0,0,0,0,0,0,0,0,0,361
10,0,0,8,1,1,0,41,1320,62,0,...,0,0,1,0,0,0,0,0,0,1464
11,0,0,0,0,0,0,2,202,405,0,...,0,0,0,0,0,0,0,0,0,618


In [58]:
pd.crosstab(pd.Series(np_utils.categorical_probas_to_classes(y_test_ary)), pd.Series(predictions.ravel()), rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0.0,1.0,All
True,,,
0,78363,103,78466
1,102,1493,1595
All,78465,1596,80061


### Testing

In [102]:
#model.get_config()
def prediction_to_str(clf_prediction, category_id):
    if(clf_prediction > 0.5):
        return str(category_id)
    else:
        return 'not ' + str(category_id)

def predict(description_str, tok_, clf_, max_length_, category_id_):
    #seq_ = tok_.texts_to_sequences([description_str])
    seq_ = texts_to_sequences_custom([description_str.lower()], tok_.word_index)
    seq_pad = sequence.pad_sequences(seq_, maxlen = max_length_)
    clf_prediction = clf_.predict(seq_pad)
    
    #print(seq_)
    #print(seq_pad)
    
    # Prediction to string
    clf_prediction_str = prediction_to_str(clf_prediction[0][0], category_id_)
    
    return clf_prediction_str
    #return clf_prediction[0][0]

def predict_2(description_str, tok_, clf_, max_length_, category_id_):
    #seq_ = tok_.texts_to_sequences([description_str])
    seq_ = texts_to_sequences_custom([description_str.lower()], tok_.word_index)
    seq_pad = sequence.pad_sequences(seq_, maxlen = max_length_)
    #clf_prediction = clf_.predict(seq_pad)
    clf_prediction = model.predict_classes(seq_pad)
    
    #print(seq_)
    #print(seq_pad)
    
    # Prediction to string
    #clf_prediction_str = prediction_to_str(clf_prediction[0][0], category_id_)
    clf_prediction = le.inverse_transform(clf_prediction)
    
    if(clf_prediction == ['Positive']):
        return str(category_id_)
    else:
        return 'not ' + str(category_id_)
    
    
def predict_proba(description_str, tok_, clf_, max_length_):
    #seq_ = tok_.texts_to_sequences([description_str])
    seq_ = texts_to_sequences_custom([description_str], tok_.word_index)
    seq_pad = sequence.pad_sequences(seq_, maxlen = max_length_)
    clf_prediction_proba = clf_.predict_proba(seq_pad, verbose=0)
    
    return clf_prediction_proba[0][0]


# id_ = 'table Setr'
# p = predict(id_, best_model_aux['Tokenizer'], best_model, best_model_aux['Max length'], best_model_aux['Category ID'])
# pp = predict_proba(id_, best_model_aux['Tokenizer'], best_model, best_model_aux['Max length'])
# print(p)
# print(pp)

In [103]:
# TEST

## load old model
model_file = "category_927_nets__traindata5000_vectrain5000_model.h5"
aux_file = "category_927_nets__traindata5000_vectrain5000_aux.pkl"
old_best_model_ = load_model(model_file)
old_best_model_aux_ = get_model_file_aux(aux_file)
old_tok_ = old_best_model_aux_['Tokenizer']
old_word_index_ = old_best_model_aux_['Tokenizer'].word_index

## use fresh model
best_model_ = best_model
best_model_aux_ = best_model_aux
tok_ = tok
word_index_ = word_index

item_d = 'NieR: Automata™ DEMO 120161128 (Playable Demo)'

# screwdrivers check
scrw_items = [
"tekton 2655 flare nut wrench set metric 6piece"
,"tekton 2780 10slot screwdriver holder and organizer"
,"titan 17237 insulated electrical screwdriver set  7 piece"
,"tool sorter screwdriver organizer red"
,"torin sdh15rt magnetic screwdriver holder"  #wrong predict
,"wera 05020013001 joker combination wrenchset 11 pieces"
,"wera kk vde 60i62i68i18 insulated pouch set with interchangeable blades 18piece" # tricky
,"wiha 28103 magnetic 14 bit holder stubby 57mm pliers screwdriver" # tricky, wrong predict
]

for n, i in enumerate(scrw_items):
    item_d = i
    
    print(str(n) + ' ' + '='*100)
    
    print('Old model prediction:')
    print('item:',item_d)
    print('Seq max len:', old_best_model_aux_['Max length'])
    print(predict(item_d, old_tok_, old_best_model_, old_best_model_aux_['Max length'], '927'))
    print(predict_proba(item_d, old_tok_, old_best_model_, old_best_model_aux_['Max length']))


    print('\nFresh model prediction:')
    print('item:',item_d)
    print('Seq max len:', best_model_aux_['Max length'])
    print(predict_2(item_d, tok_, best_model_, best_model_aux_['Max length'], '927'))
    #print(predict_proba(item_d, tok_, best_model_, best_model_aux_['Max length']))

    print()

    #tt = train_df.loc[0:10,['description_mod1']]
    #tt['pred'] = tt['description_mod1'].apply(lambda x: predict(x, best_model_aux_['Tokenizer'], best_model, best_model_aux_['Max length'], best_model_aux_['Category ID']))
    #tt['prob'] = tt['description_mod1'].apply(lambda x: predict_proba(x, best_model_aux_['Tokenizer'], best_model, best_model_aux_['Max length']))

0 ====================================================================================================
Old model prediction:
item: tekton 2655 flare nut wrench set metric 6piece
Seq max len: 30
not 927
4.17323e-08

Fresh model prediction:
item: tekton 2655 flare nut wrench set metric 6piece
Seq max len: 30
1/1 [==============================] - 0s
not 927

1 ====================================================================================================
Old model prediction:
item: tekton 2780 10slot screwdriver holder and organizer
Seq max len: 30
not 927
0.121584

Fresh model prediction:
item: tekton 2780 10slot screwdriver holder and organizer
Seq max len: 30
1/1 [==============================] - 0s
927

2 ====================================================================================================
Old model prediction:
item: titan 17237 insulated electrical screwdriver set  7 piece
Seq max len: 30
927
0.998989

Fresh model prediction:
item: titan 17237 insulated electric